### 🎯 Module Overview
This module covers everything you need to know about parsing and ingesting data for RAG systems, from basic text files to complex PDFs and databases. We'll use LangChain v0.3 and explore each technique with practical examples.

Table of Contents

- Introduction to Data Ingestion
- Text Files (.txt)
- PDF Documents
- Microsoft Word Documents
- CSV and Excel Files
- JSON and Structured Data
- Web Scraping
- Databases (SQL)
- Audio and Video Transcripts
- Advanced Techniques
- Best Practices

### Introduction to Data Ingestion

In [2]:
import os
from typing import List, Dict, Any
import pandas as pd

In [5]:
from langchain_core.documents import Document
from langchain_text_splitters import (
    RecursiveCharacterTextSplitter, 
    CharacterTextSplitter, 
    TokenTextSplitter
)
print("Setup completed !")

Setup completed !


### Understanding Document Structure in Langchain

In [ ]:
# creating a simple document
doc_sample = Document(
    page_content = "LangChain is a framework for developing applications powered by language models. It can be used for chatbots, Generative Question-Answering (GQA), summarization, and much more.",
    metadata = {
        "source": "example.txt",
        "page": 1,
        "author": "Praveen Shekarappa",
        "date_created": "04-01-2026",
        "custom_field": "Any Value"
    }
)
print("Document structure:")
print(f"content: {doc_sample.page_content}")
print(f"metadata: {doc_sample.metadata}")



Document structure:
content: LangChain is a framework for developing applications powered by language models. It can be used for chatbots, Generative Question-Answering (GQA), summarization, and much more.
metadata: {'source': 'example.txt', 'page': 1, 'author': 'Praveen Shekarappa', 'date_created': '04-01-2026', 'custom_field': 'Any Value'}


### why metadata is important?
#### Metadata provides context about the document, such as its source, author, creation date, and other relevant information. This context can be crucial for various reasons:
1. Source Tracking: Knowing where the document came from helps in assessing its credibility and reliability.
2. Contextual Understanding: Metadata can provide additional context that aids in interpreting the content correctly.
3. Search and Retrieval: Metadata can be used to index and search documents more effectively.
4. Filtering: Metadata allows for filtering documents based on specific criteria, such as date range, author, or source
5. Compliance and Auditing: In regulated industries, metadata can help track document history for compliance purposes.

In [11]:
type(doc_sample)

langchain_core.documents.base.Document

### Simplest case on Text files (.txt)

In [13]:
# create simple text file
import os
os.makedirs("data/text_files", exist_ok=True)

In [ ]:
sample_texts={
    "data/text_files/python_intro.txt":"""Python Programming Introduction

Python is a high-level, interpreted programming language known for its simplicity and readability.
Created by Guido van Rossum and first released in 1991, Python has become one of the most popular
programming languages in the world.

Key Features:
- Easy to learn and use
- Extensive standard library
- Cross-platform compatibility
- Strong community support

Python is widely used in web development, data science, artificial intelligence, and automation.""",
    
"data/text_files/machine_learning.txt": """Machine Learning Basics

Machine learning is a subset of artificial intelligence that enables systems to learn and improve
from experience without being explicitly programmed. It focuses on developing computer programs
that can access data and use it to learn for themselves.

Types of Machine Learning:
1. Supervised Learning: Learning with labeled data
2. Unsupervised Learning: Finding patterns in unlabeled data
3. Reinforcement Learning: Learning through rewards and penalties

Applications include image recognition, speech processing, and recommendation systems
    
    
    """

}

for filepath,content in sample_texts.items():
    with open(filepath,'w',encoding="utf-8") as f:
        f.write(content)

print("✅ Sample text files created!")

✅ Sample text files created!


### TextLoader - Read Single File

In [2]:
from langchain_community.document_loaders import TextLoader
# Load text files from the directory - read single file
loader = TextLoader(file_path="data/text_files/python_intro.txt", encoding="utf-8")
documents = loader.load()
# print(type(documents))
# print(documents)
print(f"Loaded {len(documents)} documents")
print(f"Content preview: {documents[0].page_content[:100]}...")
print(f"Metadata: {documents[0].metadata}")

Loaded 1 documents
Content preview: Python Programming Introduction

Python is a high-level, interpreted programming language known for ...
Metadata: {'source': 'data/text_files/python_intro.txt'}


### Directory Loader - Multiple text files

In [3]:
from langchain_community.document_loaders import DirectoryLoader
# Load all text files from the directory
dir_loader = DirectoryLoader(
    path = "data/text_files",
    glob = "**/*.txt", # pattern to match text files
    loader_cls = TextLoader, # specify the loader class
    loader_kwargs = {"encoding": "utf-8"},
    show_progress = True
)

all_documents = dir_loader.load()
print(f"Total documents loaded from directory: {len(all_documents)}")
for i, doc in enumerate(all_documents):
    print(f"\nDocument {i+1}:")
    print(f"  Source: {doc.metadata['source']}")
    print(f"  Length: {len(doc.page_content)} characters")

# 📊 Analysis
print("\n📊 DirectoryLoader Characteristics:")
print("✅ Advantages:")
print("  - Loads multiple files at once")
print("  - Supports glob patterns")
print("  - Progress tracking")
print("  - Recursive directory scanning")

print("\n❌ Disadvantages:")
print("  - All files must be same type")
print("  - Limited error handling per file")
print("  - Can be memory intensive for large directories")


100%|██████████| 2/2 [00:00<00:00, 1247.75it/s]

Total documents loaded from directory: 2

Document 1:
  Source: data/text_files/python_intro.txt
  Length: 489 characters

Document 2:
  Source: data/text_files/machine_learning.txt
  Length: 575 characters

📊 DirectoryLoader Characteristics:
✅ Advantages:
  - Loads multiple files at once
  - Supports glob patterns
  - Progress tracking
  - Recursive directory scanning

❌ Disadvantages:
  - All files must be same type
  - Limited error handling per file
  - Can be memory intensive for large directories


### Text Splitting Statergies

In [8]:
# import different text splitters
from langchain_text_splitters import (
    RecursiveCharacterTextSplitter, 
    CharacterTextSplitter, 
    TokenTextSplitter
)
print(all_documents)


[Document(metadata={'source': 'data/text_files/python_intro.txt'}, page_content='Python Programming Introduction\n\nPython is a high-level, interpreted programming language known for its simplicity and readability.\nCreated by Guido van Rossum and first released in 1991, Python has become one of the most popular\nprogramming languages in the world.\n\nKey Features:\n- Easy to learn and use\n- Extensive standard library\n- Cross-platform compatibility\n- Strong community support\n\nPython is widely used in web development, data science, artificial intelligence, and automation.'), Document(metadata={'source': 'data/text_files/machine_learning.txt'}, page_content='Machine Learning Basics\n\nMachine learning is a subset of artificial intelligence that enables systems to learn and improve\nfrom experience without being explicitly programmed. It focuses on developing computer programs\nthat can access data and use it to learn for themselves.\n\nTypes of Machine Learning:\n1. Supervised Learn

In [16]:
# Method 1: using character-based text splitter
text = all_documents[1].page_content
print(f"Original Text Length: {len(text)} characters")
char_splitter = CharacterTextSplitter(
    separator = "\n", # specify separator, split by new line
    chunk_size = 200, # chunk size
    chunk_overlap = 20, # chunk overlap, number of characters to overlap between chunks
    length_function = len # function to measure length (here, number of characters
)
char_chunks = char_splitter.split_text(text)
print(f"Total Chunks Created: {len(char_chunks)}")
print(f"print first chunk:\n{char_chunks[0][:100]}...")
print(char_chunks)

Original Text Length: 575 characters
Total Chunks Created: 4
print first chunk:
Machine Learning Basics
Machine learning is a subset of artificial intelligence that enables systems...
['Machine Learning Basics\nMachine learning is a subset of artificial intelligence that enables systems to learn and improve', 'from experience without being explicitly programmed. It focuses on developing computer programs\nthat can access data and use it to learn for themselves.\nTypes of Machine Learning:', '1. Supervised Learning: Learning with labeled data\n2. Unsupervised Learning: Finding patterns in unlabeled data\n3. Reinforcement Learning: Learning through rewards and penalties', 'Applications include image recognition, speech processing, and recommendation systems']


In [18]:
print(char_chunks[0])
print("-----")
print(char_chunks[1])
print("-----")
print(char_chunks[2])

Machine Learning Basics
Machine learning is a subset of artificial intelligence that enables systems to learn and improve
-----
from experience without being explicitly programmed. It focuses on developing computer programs
that can access data and use it to learn for themselves.
Types of Machine Learning:
-----
1. Supervised Learning: Learning with labeled data
2. Unsupervised Learning: Finding patterns in unlabeled data
3. Reinforcement Learning: Learning through rewards and penalties


In [19]:
# Method 2: using recursive character text splitter
recursive_splitter = RecursiveCharacterTextSplitter(
    separators = ["\n\n", "\n", " ", ""], # list of separators
    chunk_size = 200,
    chunk_overlap = 20,
    length_function = len
)
recursive_chunks = recursive_splitter.split_text(text)
print(f"Total Chunks Created (Recursive): {len(recursive_chunks)}")
print(f"First Chunk (Recursive):\n{recursive_chunks[0][:100]}...")


Total Chunks Created (Recursive): 6
First Chunk (Recursive):
Machine Learning Basics...


In [21]:
print(recursive_chunks[0])
print("--------")
print(recursive_chunks[1])
print("--------")
print(recursive_chunks[2])
print("--------")
print(recursive_chunks[3])

Machine Learning Basics
--------
Machine learning is a subset of artificial intelligence that enables systems to learn and improve
from experience without being explicitly programmed. It focuses on developing computer programs
--------
that can access data and use it to learn for themselves.
--------
Types of Machine Learning:
1. Supervised Learning: Learning with labeled data
2. Unsupervised Learning: Finding patterns in unlabeled data
